## Interpteter for Boolean Expressions

Install the lark-parser with

> pip install lark-parser 

### Grammar

In [1]:
from lark import Lark, InlineTransformer 

grammar_boolexp = """
    ?start: boolexp
    ?boolexp: booland
        | boolexp "or" booland -> orop
    ?booland: boolatom
        | booland "and" boolatom -> andop
    ?boolatom: "True"       -> true
        | "False"           -> false
        | "not" boolatom    -> notop
        | NAME             -> var
        | "(" boolexp ")"
    %import common.CNAME -> NAME
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

### Parser
Implementing the functions described by grammar productions

In [2]:
class Tree_BoolExp(InlineTransformer):

    def var(self, name):
        return ['variable', str(name)]

    def false(self):
        return ['bool_const', False]
    
    def true(bool_const):
        return ['bool_const', True]
    
    def orop(self, left, right):
        return ['or', left, right]
    
    def andop(self, left, right):
        return ['and', left, right]

    def notop(self, value):
        return ['not', value]

Building the parser using the defined AST representation

In [3]:
def get_parser_boolexp():
    parser = Lark(grammar_boolexp, parser='lalr', transformer=Tree_BoolExp())
    return parser.parse

Example of usage

In [11]:
parse = get_parser_boolexp()
B1 = parse("True or False")
B1

['or', ['bool_const', True], ['bool_const', False]]

In [8]:
B2 = parse("(x or y) and not False")
B2

['and',
 ['or', ['variable', 'x'], ['variable', 'y']],
 ['not', ['bool_const', False]]]

In [10]:
B3 = parse("not (x or y) and z")
B3

['and',
 ['not', ['or', ['variable', 'x'], ['variable', 'y']]],
 ['variable', 'z']]

### Interpreter

- Objective: define solve_bool(E, sigma)

How to get to a function that given an AST and a sigma evaluate the Boolean Expression in sigma?

In [12]:
def solve_bool(exp, sigma):
    op = exp[0]
    if op=='bool_const':
        return exp[1]
    if op=='variable':
        return sigma[exp[1]]
    if op=='or':
        """ lazy evaluation - default in Python """
        return solve_bool(exp[1], sigma) or solve_bool(exp[2], sigma)
        """ Eager evaluation 
        left = solve_bool(exp[1], sigma)
        right = solve_bool(exp[2], sigma)
        return left or right
        """
    if op=='and':
        """ lazy evaluation - default in Python """
        return solve_bool(exp[1], sigma) and solve_bool(exp[2], sigma)
        """ Eager evaluation 
        left = solve_bool(exp[1], sigma)
        right = solve_bool(exp[2], sigma)
        return left and right
        """
    if op=='not':
        return not solve_bool(exp[1], sigma)

Evaluation Examples

In [13]:
sigma = {} # empty interpretation
print('B1 = ', solve_bool(B1, sigma))

B1 =  True


In [14]:
sigma = { 'x':False, 'y':True }
print('B2 = ', solve_bool(B2, sigma))

B2 =  True


In [15]:
sigma = { 'x':False, 'y':True, 'z':False }
print('B3 = ', solve_bool(B3, sigma))

B3 =  False
